In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore") 

os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [2]:
# boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2021-01-06 (5).xlsx")
boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
boilings = make_boilings_by_groups(boiling_plan_df)
schedule = make_schedule(boilings, start_times={'water': '08:00', 'salt': '07:00'})
frontend = make_frontend(schedule)
# draw_excel_frontend(frontend, open_file=True)

In [3]:
df = boiling_plan_df.copy()

In [4]:
mark_consecutive_groups(df, 'boiling', 'boiling_group')
df.head()

,batch_id,boiling_params,sku,kg,packing_team_id,boiling_line_type,boiling,boiling_group
0,81,"3.3, Сакко",<SKU 24>,99,1,water,<Boiling 6>,1
1,81,"3.3, Сакко",<SKU 27>,302,1,water,<Boiling 6>,1
2,81,"3.3, Сакко",<SKU 22>,33,1,water,<Boiling 6>,1
3,81,"3.3, Сакко",<SKU 23>,243,1,water,<Boiling 6>,1
4,81,"3.3, Сакко",<SKU 26>,103,1,water,<Boiling 6>,1


In [5]:
boiling_group_df = df[df['boiling_group'] == 2]
boiling_group_df

,batch_id,boiling_params,sku,kg,packing_team_id,boiling_line_type,boiling,boiling_group
7,82,"3.6, Альче",<SKU 12>,26,1,water,<Boiling 9>,2
8,82,"3.6, Альче",<SKU 25>,974,1,water,<Boiling 9>,2
9,83,"3.6, Альче",<SKU 30>,1000,1,water,<Boiling 9>,2
10,84,"3.6, Альче",<SKU 30>,1000,1,water,<Boiling 9>,2


In [6]:
def cast_bff(sku):
    if sku == cast_sku(4):
        return cast_boiling_form_factor(5)
    elif sku == cast_sku(37):
        return cast_boiling_form_factor(8)
    else:
        return sku.boiling_form_factors[0]

In [7]:
ERROR = 1e-5

class BoilingGroupToSchemaTransformer:
    def _calc_melting_speed(self, boiling_group_df):
        boiling_type = df.iloc[0]['boiling'].boiling_type
        return 900 if boiling_type == 'water' else 1020 # todo: make properly
    
    def _calc_boilings_meltings(self, boiling_group_df):
        df = boiling_group_df.copy()
        
        # todo: take from boiling_plan
        df['bff'] = df['sku'].apply(cast_bff)
        df['bff_id'] = df['bff'].apply(lambda bff: bff.id)
        df = df.reset_index()

        bff_kgs = df.groupby('bff_id').agg({'kg': 'sum', 'bff': 'first', 'index': 'first'})
        bff_kgs = bff_kgs.sort_values(by='index')
        
        iterator = SimpleBoundedIterator([[row['bff'], row['kg']] for bff_id, row in bff_kgs.iterrows()]) 
        
        boiling_type = df.iloc[0]['boiling'].boiling_type
        boiling_volume = 1000 if boiling_type == 'water' else 850 # todo: make properly
        total_kg = df['kg'].sum()
        
        assert total_kg % boiling_volume == 0
        
        boiling_volumes = [boiling_volume] * int(total_kg // boiling_volume)
        
        res = []
        for boiling_volume in boiling_volumes:
            boiling_meltings = []

            left = boiling_volume

            while left > ERROR:
                bff, left_kg = iterator.current()

                cur_value = min(left, left_kg)

                iterator.current()[-1] -= cur_value
                left -= cur_value

                boiling_meltings.append([bff, cur_value])

                if abs(iterator.current()[-1]) < ERROR:
                    iterator.next()

            res.append(boiling_meltings)
        return res
    
    def _calc_packings(self, boiling_group_df):
        df = boiling_group_df.copy()
        
        # todo: take from boiling_plan
        df['bff'] = df['sku'].apply(cast_bff)
        
        df['sku_id'] = df['sku'].apply(lambda sku: sku.id)
        df = df.reset_index()

        sku_kgs = df.groupby(['packing_team_id', 'sku_id']).agg({'kg': 'sum', 'sku': 'first', 'bff': 'first', 'index': 'first'})
        sku_kgs = sku_kgs.sort_values(by='index').reset_index()

        res = {}

        for packing_team_id, grp in sku_kgs.groupby('packing_team_id'):
            res[packing_team_id] = []

            for _, row in grp.iterrows():
                res[packing_team_id].append([row['sku'], row['bff'], row['kg']])
        return res
    
    def __call__(self, boiling_group_df):
        return self._calc_boilings_meltings(boiling_group_df), self._calc_packings(boiling_group_df), self._calc_melting_speed(boiling_group_df)

In [8]:
transformer = BoilingGroupToSchemaTransformer()
boilings_meltings, packings, melting_speed = transformer(boiling_group_df)
boilings_meltings, packings, melting_speed

([[[<BoilingFormFactor 3>, 26], [<BoilingFormFactor 13>, 974]],
  [[<BoilingFormFactor 4>, 1000]],
  [[<BoilingFormFactor 4>, 1000]]],
 {1: [[<SKU 12>, <BoilingFormFactor 3>, 26],
   [<SKU 25>, <BoilingFormFactor 13>, 974],
   [<SKU 30>, <BoilingFormFactor 4>, 2000]]},
 900)

In [9]:
cast_sku(1).boilings[0]

<Boiling 2>

In [14]:
cast_bff(cast_sku(1)).post_processing_time

90.0

In [26]:
# generate meltings by boilings
res = []
for boiling_meltings in boilings_meltings:
    boiling_volume = sum(x[1] for x in boiling_meltings)
    
    drenator = Container('Drenator', value=boiling_volume, max_pressures=[None, None])
    
    melting_processors = []
    cooling_processors = []
    for i, (bff, kg) in enumerate(boiling_meltings):
        melting_processor = Processor(f'Melting{i}', 
                                      items=['cheese_mass', bff],  
                                      max_pressures=[melting_speed, melting_speed], 
                                      processing_time=0,  
                                      limits=[kg, kg])
        melting_processors.append(melting_processor)
        
        cooling_processor = Processor(f'Cooling{i}', 
                                      items=[bff, bff], 
                                      max_pressures=[None, None], 
                                      processing_time=bff.post_processing_time / 60, 
                                      limits=[kg, kg])
        cooling_processors.append(cooling_processor)
    melting_queue = Queue('MeltingQueue', melting_processors, break_funcs={'in': lambda old, new: 1 / 12})
    cooling_queue = Queue('CoolingQueue', cooling_processors)
    
    res.append([drenator, melting_queue, cooling_queue])
res

[[<utils_ak.fluid_flow.actors.container.Container at 0x15c9e1a36d0>,

In [27]:
# generate packing queues
packing_queues = []
for packing_team_id, grp in sorted(packings.items(), key=lambda v: v[0]):
    processors = []
    
    for j, (sku, bff, kg) in enumerate(grp):
        processor = Processor(f'Packing{j}', 
                              items=[bff, sku],
                             max_pressures=[sku.packing_speed, None],
                             processing_time=0,
                             limits=[kg, None])
        processors.append(processor)
    packing_queue = Queue(f'PackingQueue{packing_team_id}', processors)
    packing_queues.append(packing_queue)
packing_queues

In [28]:
for drenator, melting_queue, cooling_queue in res:
    packing_hub = Hub('hub')

    pipe_connect(drenator, melting_queue, 'drenator-melting')
    pipe_connect(melting_queue, cooling_queue, 'melting-cooling')
    pipe_connect(cooling_queue, packing_hub, 'cooling-hub')
    
    for i, packing_queue in enumerate(packing_queues):
        pipe_connect(packing_hub, packing_queue, f'hub-packing_queue{i}')

    flow = FluidFlow(drenator, verbose=True)
    run_flow(flow)

    maker, make = init_block_maker('root', axis=1)

    for node in drenator.iterate('down'):
        if node.active_periods():
            for period in node.active_periods():
                label = '-'.join([str(node.id), str(period[0])])
                beg, end = period[1:]
                beg, end = custom_round(beg * 60, 5, 'ceil') // 5, custom_round(end * 60, 5, 'ceil') // 5
                make(label, x=[beg, 0], size=(end - beg, 1))
    print(maker.root.tabular())

    # clean-up
    for node in drenator.iterate('down'):
        node.reset()

    # remove packers from current boiling
    for packing_queue in packing_queues:
        pipe_disconnect(packing_hub, packing_queue)

Processing time 0
Flow:
    Container Drenator:default: 1000
    Queue: MeltingQueue: [["Melting0", [0, 0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Processing time 0.028888888888888888
Flow:
    Container Drenator:default: 974.0
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [26.0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Resuming 0.11222222222222222
Processing time 0.11222222222222222
Flow:
    Container Drenator:default: 974.0
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]], ["Melting1", [0, 0]]]
    Queue: CoolingQueue: [["Cooling0", [26.0, 0]], ["Cooling1", [0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0, 0]], ["Packing1", [0, 0]], ["Packing2", [0, 0]]]

Processing time 1.111111111111111

Flow:
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]]]
    Queue: CoolingQueue: [["Cooling0", [1000.0, 0]]]
    Queue: PackingQueue1: [["Packing0", [0.0, 25.999999999999908]], ["Packing1", [0.0, 974.0]], ["Packing2", [0.0, 999.9999999999999]]]

Processing time 2.361111111111111
Flow:
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]]]
    Queue: CoolingQueue: [["Cooling0", [0.0, 26.66666666666663]]]
    Queue: PackingQueue1: [["Packing0", [0.0, 25.999999999999908]], ["Packing1", [0.0, 974.0]], ["Packing2", [0.0, 1973.3333333333333]]]

Processing time 2.361111111111111
Flow:
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]]]
    Queue: CoolingQueue: [["Cooling0", [0.0, 26.66666666666663]]]
    Queue: PackingQueue1: [["Packing0", [0.0, 25.999999999999908]], ["Packing1", [0.0, 974.0]], ["Packing2", [0.0, 1973.3333333333333]]]

Processing time 2.391552511415525
Flow:
    Queue: MeltingQueue: [["Melting0", [0.0, 0.0]]]
    Queue: CoolingQueue: [["Cooling0", [0.0, 1.5631940186722204e-13]